In [ ]:
!pip install transformers datasets torchaudio soundfile kenlm pyctcdecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 M

In [ ]:
import shutil

# Xóa folder cũ nếu có
if os.path.exists(extract_dir):
    shutil.rmtree(extract_dir)

# Tạo folder mới
os.makedirs(extract_dir, exist_ok=True)


NameError: name 'os' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import os
DATA_DIR = "/content/drive/My Drive/VLSP2025/"
ZIP_FILE = os.path.join(DATA_DIR, "wav-private.zip")
WAV_DIR = os.path.join(DATA_DIR, "wav")
RESULT_FILE = os.path.join(DATA_DIR, "results.tsv")
MODEL_ID = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"
LM_PATH = os.path.join(DATA_DIR, "vi_lm_4grams.bin")

# =======================
# Chuẩn hóa văn bản
# =======================
RE_REPLACE_PATTERN_MAP = {
    re.compile(r"\b([hkltv])y\b"): r"\1i",
    re.compile(r"\b([bdđhklmnprstvx])ỳ\b"): r"\1ì",
    re.compile(r"\b([bdđhklmnprstvx])ý\b"): r"\1í",
    re.compile(r"\b([bdđhklmnprstvx])ỷ\b"): r"\1ỉ",
    re.compile(r"\b([bdđhklmnprstvx])ỹ\b"): r"\1ĩ",
    re.compile(r"\b([bdđhklmnprstvx])ỵ\b"): r"\1ị",
}

def norm_transcript(text: str):
    text = " " + text.strip() + " "
    text = text.replace("quy ", "qui ")
    text = text.replace("quỳ ", "quì ")
    text = text.replace("quý ", "quí ")
    text = text.replace("quỹ ", "quĩ ")
    text = text.replace("quỵ ", "quỵ ")
    for pattern, repl in RE_REPLACE_PATTERN_MAP.items():
        text = re.sub(pattern, repl, text)
    return text.strip()


In [ ]:
import os
DATA_DIR = "/content/drive/My Drive/VLSP2025/"
print(os.listdir(DATA_DIR))  # liệt akê file/folder trong DATA_DIR


['wav-public.zip', 'SER.csv', 'results.tsv', 'results.gsheet', 'vi_lm_4grams.bin', 'wav', 'wav-private.zip', 'results_new.tsv']


In [ ]:
import os

def dem_so_file_wav(folder):
    """Đếm số lượng file .wav trong folder và các subfolder"""
    count = 0
    for root, dirs, files in os.walk(folder):
        count += sum(1 for f in files if f.lower().endswith('.wav'))
    return count

# Ví dụ sử dụng
WAV_DIR = "/content/drive/My Drive/VLSP2025/wav"
so_file = dem_so_file_wav(WAV_DIR)
print(f"Tổng số file .wav trong folder: {so_file}")


Tổng số file .wav trong folder: 4693


In [ ]:
# =======================
# Import
# =======================
import os
import zipfile
import re
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from pyctcdecode import build_ctcdecoder

# =======================
# Cấu hình
# =======================
DATA_DIR = "/content/drive/My Drive/VLSP2025/"
ZIP_FILE = os.path.join(DATA_DIR, "wav-private.zip")
WAV_DIR = os.path.join(DATA_DIR, "wav")
RESULT_FILE = os.path.join(DATA_DIR, "results_new.tsv")
MODEL_ID = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"
LM_PATH = os.path.join(DATA_DIR, "vi_lm_4grams.bin")

In [ ]:

# =======================
# Load model và processor
# =======================
print("Tải model và processor...")
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# =======================
# Tạo decoder với LM 4-grams (fix vocab size)
# =======================
labels = processor.tokenizer.convert_ids_to_tokens(range(model.config.vocab_size))
decoder = build_ctcdecoder(labels=labels, kenlm_model_path=LM_PATH)


Tải model và processor...


In [ ]:

# =======================
# Hàm nhận dạng 1 file wav
# =======================
def transcribe(file_path):
    # Load audio
    speech, rate = torchaudio.load(file_path)
    if rate != 16000:
        speech = torchaudio.transforms.Resample(orig_freq=rate, new_freq=16000)(speech)
    input_values = processor(speech.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_values
    input_values = input_values.to(device)

    # Forward model
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode với LM 4-grams
    transcription = decoder.decode(logits.cpu().numpy()[0])
    return transcription.lower()


In [ ]:
import os
from zipfile import ZipFile

DATA_DIR = "/content/drive/My Drive/VLSP2025/"
ZIP_PRIVATE = os.path.join(DATA_DIR, "wav-private.zip")
WAV_PRIVATE_DIR = os.path.join(DATA_DIR, "wav_private")

os.makedirs(WAV_PRIVATE_DIR, exist_ok=True)

with ZipFile(ZIP_PRIVATE, 'r') as zip_ref:
    zip_ref.extractall(WAV_PRIVATE_DIR)

print("Đã giải nén xong vào:", WAV_PRIVATE_DIR)


Đã giải nén xong vào: /content/drive/My Drive/VLSP2025/wav_private


In [ ]:
import os

# Thư mục chứa file private
WAV_PRIVATE_DIR = "/content/drive/My Drive/VLSP2025/wav_private"

# File private đầu tiên cần kiểm tra
file_check = "vlsp_private_0001.wav"
file_path = os.path.join(WAV_PRIVATE_DIR, file_check)

if os.path.exists(file_path):
    print(f"File {file_check} đã tồn tại trong thư mục private.")
else:
    print(f"File {file_check} **chưa có** trong thư mục private.")

# Kiểm tra tổng số file .wav private
def dem_so_file_wav(folder):
    count = 0
    for root, dirs, files in os.walk(folder):
        count += sum(1 for f in files if f.lower().endswith('.wav'))
    return count

so_file = dem_so_file_wav(WAV_PRIVATE_DIR)
print(f"Tổng số file .wav private hiện có: {so_file}")


File vlsp_private_0001.wav **chưa có** trong thư mục private.
Tổng số file .wav private hiện có: 2349


In [ ]:
import os
from zipfile import ZipFile

# Đường dẫn file zip private
DATA_DIR = "/content/drive/My Drive/VLSP2025/"
ZIP_PRIVATE = os.path.join(DATA_DIR, "wav-private.zip")

# Mở zip và lấy danh sách file
with ZipFile(ZIP_PRIVATE, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    print(f"Tổng số file trong zip: {len(file_list)}")

    print("\n10 file đầu trong zip:")
    for i, fname in enumerate(file_list[:10], 1):
        print(f"{i}: {fname}")


Tổng số file trong zip: 2350

10 file đầu trong zip:
1: wav/
2: wav/vlsp2025_private_1753.wav
3: wav/vlsp2025_private_0670.wav
4: wav/vlsp2025_private_1998.wav
5: wav/vlsp2025_private_1288.wav
6: wav/vlsp2025_private_0060.wav
7: wav/vlsp2025_private_1278.wav
8: wav/vlsp2025_private_0437.wav
9: wav/vlsp2025_private_0050.wav
10: wav/vlsp2025_private_1614.wav


In [ ]:
import os

# Folder chứa file private (sau khi giải nén)
private_wav_folder = os.path.join(WAV_PRIVATE_DIR, "wav")

# Lấy danh sách file WAV private hiện có
wav_files = sorted([
    f for f in os.listdir(private_wav_folder)
    if f.endswith(".wav") and f.startswith("vlsp2025_private_")
])

if not wav_files:
    print("Chưa có file private nào trong thư mục.", private_wav_folder)
else:
    with open(RESULT_FILE, "w", encoding="utf-8") as fw:
        for idx, wav_file in enumerate(wav_files, 1):
            file_path = os.path.join(private_wav_folder, wav_file)
            text = transcribe(file_path)
            text_norm = norm_transcript(text)
            fw.write(f"{wav_file}\tneutral\t{text_norm}\n")

            if idx % 50 == 0 or idx == len(wav_files):
                print(f"[{idx}/{len(wav_files)}] {wav_file} -> {text_norm}")

print(f"\nHoàn thành! File kết quả: {RESULT_FILE}")


[50/2349] vlsp2025_private_0050.wav -> có mỗi chuyện đi ăn nhà hàng thôi mà cãi nhau một tiếng rưỡi đồng hồ vẫn chưa xong
[100/2349] vlsp2025_private_0100.wav -> tới ngày bao nhiêu người tại sao chẳng đúng khách cuối tuần bắt là sao thôi thôi thôi chú tư chú đừng có chửi nữa con tặng chú sứ đâu lành nè chú cho mét người nha
[150/2349] vlsp2025_private_0150.wav -> là anh đã làm cho tôi trở nên đáng thương trong mắt bố mẹ anh
[200/2349] vlsp2025_private_0200.wav -> và một phần ấp thới tây một thuộc xã tân hiệp huyện hóc môn
[250/2349] vlsp2025_private_0250.wav -> thế nhưng mà thực chất là để lừa đảo chiếm dụng của người dân
[300/2349] vlsp2025_private_0300.wav -> mấy chồng nó cũng đi kiểm tra nhưng mà tại vì gan của nó nhỏ quá cho nên
[350/2349] vlsp2025_private_0350.wav -> còn hợp tác xã thì nó là một cái tập thể
[400/2349] vlsp2025_private_0400.wav -> thì chị cũng thấy đó là một cái điều hết sức đặc biệt của một người nổi tiếng nha
[450/2349] vlsp2025_private_0450.wav -> bó mới sẵn quá